## new feature test

In [30]:
import pickle
import numpy as np
import tensorflow as tf

from math import sqrt
from tqdm import tqdm
from copy import deepcopy
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [54]:
with open("raw.pickle","rb") as f:
    data = pickle.load(f)

with open("matrix.pickle","rb") as f:
    matrix = pickle.load(f)

2000


In [38]:
in_matrix = {}
for k in matrix:
    for v in matrix[k]:
        if v in in_matrix:
            in_matrix[v].add(k)
        else:
            in_matrix[v] = set([k])

In [39]:
def hop_point(a, b, in_matrix, matrix, normalize=True):
    cnt = 0
    for hop in matrix[a]:
        if hop in in_matrix[b]:
            cnt += 1
    if normalize:
        cnt /= len(matrix[a])
    return cnt

def kNN(a, b, in_matrix, matrix):
    wa_out = 1/sqrt(1+len(matrix[a]))
    wb_in = 1/sqrt(1+len(in_matrix[b]))
    return wa_out + wb_in

In [55]:
tf.reset_default_graph()

# give id to node with outdegree > 0
valid_nodes = list(matrix.keys())
v2id = {}
for idx, v in enumerate(valid_nodes):
    v2id[v] = idx
print(len(v2id))

# filter sinks with outdegree 0
m = {}
for vi in matrix:
    valid = set()
    for vj in matrix[vi]:
        if vj in v2id:
            valid.add(vj)
    m[vi] = [v2id[v] for v in valid]

print(len(m))

# computation graph for rooted page rank
d = 0.5
walk = 5
A_ = tf.placeholder(tf.float32, [len(v2id)+1, len(v2id)+1])
x_ = tf.placeholder(tf.float32, [len(v2id)+1,1])
prob = tf.matmul(A_, x_)
for i in range(walk-1):
    prob = (1 - d) * prob + d * tf.matmul(A_, prob)


def pagerank(a, b, matrix):
    
    m_ = deepcopy(m)
    
    A = np.zeros([len(v2id)+1, len(v2id)+1])
    
    # add b to last dimension
    if b not in matrix:
        for v in m_:
            if b in matrix[v]:
                m_[v].append(len(v2id))
    
    # generate transition matrix
    for vi in valid_nodes:
        if vi in m_ and len(m[vi]) > 0:
            A[v2id[vi], m_[vi]] = 1/len(m_[vi])
    
    # remove link for true label
    if b in matrix[a]:
        for i in range(len(valid_nodes)):
            if len(m[a]) > 1:
                A[v2id[a], i] = 1/(len(m[a])-1) if valid_nodes[i] in matrix[a] and valid_nodes[i] != b else 0
            else:
                A[v2id[a], i] = 0
    
    
    x = np.zeros(len(v2id)+1)
    x[v2id[a]] = 1
    """
    for i in range(walk):
        x = (1 - d) * x + d * A.T @ x
    """
    x = sess.run(prob, feed_dict={A_:A.T, x_:np.expand_dims(x,-1)})
    
    if b in matrix:
        return x[v2id[b]][0]
    else:
        return x[-1][0]

19570
19570


In [48]:
with tf.device("/gpu:0"):
    with tf.Session() as sess:
        train_ = []
        for i, a, b, label in tqdm(data, ascii=True):
            #hop = hop_point(a, b, in_matrix, matrix)
            #knn = kNN(a, b, in_matrix, matrix)
            pr = pagerank(a, b, matrix)
            train_.append([pr, label])
        train_ = np.array(train_)


100%|################################################################################| 200/200 [31:51<00:00,  9.56s/it]


In [56]:
with tf.device("/gpu:0"):
    with tf.Session() as sess:
        test_pagerank = []
        for a, b in tqdm(test_pair, ascii=True):
            #hop = hop_point(a, b, in_matrix, matrix)
            #knn = kNN(a, b, in_matrix, matrix)
            pr = pagerank(a, b, matrix)
            test_pagerank.append([pr])
        test_pagerank = np.array(test_pagerank)


100%|############################################################################| 2000/2000 [4:37:37<00:00,  8.33s/it]


In [59]:
test_pagerank.shape

(2000, 1)

In [67]:
to_disc = {}
for i in range(len(test_pagerank)):
    to_disc[i] = test_pagerank[i][0]
with open("pagerank_test.pickle","wb") as f:
    pickle.dump(to_disc, f)

In [49]:
train, test = train_test_split(train_, train_size=0.8)
X_train, y_train = train[:,:-1], train[:,-1]
X_test, y_test = test[:,:-1], test[:,-1]

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [68]:
model = LogisticRegression()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(roc_auc_score(y_test, np.squeeze(model.predict_proba(X_test)[:,1])))

0.4
0.9765625


In [51]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(roc_auc_score(y_test, np.squeeze(model.predict_proba(X_test)[:,1])))

0.925
0.9088541666666666


In [64]:
model = RandomForestClassifier(100)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(roc_auc_score(y_test, np.squeeze(model.predict_proba(X_test)[:,1])))

0.925
0.9401041666666666


In [65]:
y_pred_prob = model.predict_proba(test_pagerank)
with open("pred_pagerank.csv","w") as f:
    f.write("Id,Prediction\n")
    for i in range(1,len(y_pred_prob)+1):
        f.write(str(i) + "," + str(y_pred_prob[i-1][1]) + "\n")

In [61]:
print(train_)

[[1.18588567e-07 0.00000000e+00]
 [3.19863576e-03 1.00000000e+00]
 [5.62300775e-05 0.00000000e+00]
 [1.98891066e-05 0.00000000e+00]
 [1.20041741e-05 0.00000000e+00]
 [1.83603538e-06 0.00000000e+00]
 [6.25127601e-03 1.00000000e+00]
 [1.60831250e-02 1.00000000e+00]
 [2.90424327e-06 0.00000000e+00]
 [3.66606901e-06 0.00000000e+00]
 [8.50270386e-04 1.00000000e+00]
 [8.47878070e-08 0.00000000e+00]
 [3.21984116e-04 1.00000000e+00]
 [6.38553174e-04 1.00000000e+00]
 [1.25946791e-03 1.00000000e+00]
 [9.49896162e-07 0.00000000e+00]
 [3.46453220e-04 1.00000000e+00]
 [7.32797503e-08 0.00000000e+00]
 [2.93452118e-04 1.00000000e+00]
 [7.24976044e-03 1.00000000e+00]
 [3.35672339e-05 0.00000000e+00]
 [9.94531911e-06 0.00000000e+00]
 [2.91125703e-04 1.00000000e+00]
 [6.73288479e-03 1.00000000e+00]
 [2.42267561e-06 0.00000000e+00]
 [2.35422594e-05 0.00000000e+00]
 [5.15905231e-06 0.00000000e+00]
 [5.05344815e-06 0.00000000e+00]
 [8.18299086e-05 1.00000000e+00]
 [2.01998837e-03 1.00000000e+00]
 [1.018503